MERGE AUDIO AND VIDEO FILES

In [ ]:
import numpy as np
import pandas as pd
import math
import random
import matplotlib.pyplot as plt
import os

In [ ]:
# Merge the audio and video in the folders
import subprocess

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Script below merges the audio and video for Mahnob in order to facilitate labelling
project_home_dir = os.getcwd()

# Set the path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/data'

In [ ]:

output_folder = "/content/drive/MyDrive/data/output_files"
os.makedirs(output_folder, exist_ok=True)

#Generate the file names in the 'Sesions' folder
for directory, subdirs, files in os.walk(os.path.join(data_folder, "Sessions")):
    for subdir in subdirs:
        # Inside the session folder
        os.chdir(os.path.join(project_home_dir, directory, subdir))

        # List the audio and video files in the current directory
        audio_file = [f for f in os.listdir('.') if (os.path.isfile(f) and '.wav' in f)]
        video_file = [f for f in os.listdir('.') if (os.path.isfile(f) and '.avi' in f)]

        # Checks if there is atleast one audio or video file
        if len(audio_file) > 0 and len(video_file) > 0:
          #Create mixed file by concatenating firt elemt of audio and video file
            mixed_file = str(audio_file[0] + '_' + video_file[0].replace('.avi', '.mkv'))
            output_path = os.path.join(output_folder, mixed_file)
            #ffmpeg command combines the audio and video files into a single file using specific parameters
            cmd = f'ffmpeg -y -i "{audio_file[0]}" -r 30 -i "{video_file[0]}" -filter:a aresample=async=1 -c:a flac -c:v copy "{output_path}"'
            subprocess.run(cmd, shell=True)#executes the ffmpeg command in a shell


AUDIO FEATURE PROCESSING

In [ ]:
import numpy as np
import h5py
import pandas as pd
import math
import random
import matplotlib.pyplot as plt
import os

In [ ]:
# Audio  MFCC Package
import librosa
import soundfile as sf

In [ ]:
# Keep randomness the same
np.random.seed(0)

In [ ]:
#to adjust the display options for DataFrames when they are printed
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
#pd.set_option('display.height', 1000)
pd.set_option('display.width', 1000)

In [ ]:
# Downsample all the laughs to 2.5 seconds = (44100 samples/sec * 2.5)   @44KHz
num_samples = 110250  # Each audio file should have a length of 110,250 samples
num_MFCC = 20  # Number of MFCC features (this will result in each audio having 20x216 input shape)

In [ ]:
input_data = []

In [ ]:
# Empty Annotations File
df_annotations = pd.DataFrame(index=range(0,0), columns=['Type', 'Start Time (sec)', 'End Time (sec)'])


Section to save audio subclips - optional

In [ ]:

# # Create a folder for audio subclips if it doesn't exist
# audio_subclips_folder = os.path.join(data_folder, "audio_subclips")
# if not os.path.exists(audio_subclips_folder):
#     os.makedirs(audio_subclips_folder)

# for directory, subdirs, files in os.walk(os.path.join(data_folder, "Sessions")):
#     subdirs = [int(i) for i in subdirs]
#     subdirs.sort()
#     subdirs = [str(i) for i in subdirs]
#     for subdir in subdirs:
#         session_folder_path = os.path.join(data_folder, directory, subdir)
#         os.chdir(session_folder_path)

#         print(f"Current Working Directory: {os.getcwd()}")

#         # List the files in the current directory
#         print(f"Files in Current Directory: {os.listdir('.')}")

#         audio_files = [f for f in os.listdir('.') if (os.path.isfile(f) and f.endswith('.wav'))]

#         if not audio_files:
#             print(f"No audio files found in {session_folder_path}. Skipping.")
#             continue

#         # Create a subfolder for each session within the audio subclips folder
#         session_output_folder = os.path.join(audio_subclips_folder, f"session_{subdir}")
#         if not os.path.exists(session_output_folder):
#             os.makedirs(session_output_folder)

#         # Iterate over annotation files
#         for audio_file in audio_files:
#             # Construct the full path to the CSV file
#             csv_file_path = os.path.join(session_folder_path, 'laughterAnnotation.csv')
#             if not os.path.exists(csv_file_path):
#                 print(f"Skipping {session_folder_path} as 'laughterAnnotation.csv' does not exist.")
#                 continue

#             # Read the CSV file if it exists
#             df_annotation = pd.read_csv(csv_file_path, encoding="ISO-8859-1")
#             df_annotation = df_annotation.loc[(df_annotation['Type'] == 'Laughter') | (df_annotation['Type'] == 'PosedLaughter')].iloc[:, 1:4]

#             # Iterate over annotation rows
#             for index, row in df_annotation.iterrows():
#                 start_time = row[1]
#                 end_time = row[2]

#                 y, sr = librosa.load(audio_file, offset=start_time, duration=(end_time - start_time), sr=None)

#                 num_samples = 44100  # Example value, adjust according to your requirements
#                 if len(y) > num_samples:
#                     downsampled_audio = y[:num_samples]
#                 else:
#                     padded_zeros = np.zeros(num_samples - len(y))
#                     downsampled_audio = np.concatenate((y, padded_zeros))

#                 # Save subclip
#                 subclip_filename = f"subclip_{index}.wav"
#                 subclip_path = os.path.join(session_output_folder, subclip_filename)
#                 sf.write(subclip_path, downsampled_audio, sr)

#         os.chdir(data_folder)


Section to process .wav files

In [ ]:
for directory, subdirs, files in os.walk(os.path.join(data_folder, "Sessions")):
    subdirs = [int(i) for i in subdirs]
    subdirs.sort()
    subdirs = [str(i) for i in subdirs]
    for subdir in subdirs:
      #changes the current working dir to the path of the current session folder
        session_folder_path = os.path.join(data_folder, directory, subdir)
        os.chdir(session_folder_path)

        print(f"Current Working Directory: {os.getcwd()}")

        # List the files in the current directory
        print(f"Files in Current Directory: {os.listdir('.')}")
        #Looks for audio files with .wav extension
        audio_file = [f for f in os.listdir('.') if (os.path.isfile(f) and '.wav' in f)]

        # Construct the full path to the CSV file
        csv_file_path = os.path.join(session_folder_path, 'laughterAnnotation.csv')
        if 'laughterAnnotation.csv' in os.listdir('.'):
            csv_file_path = os.path.join(session_folder_path, 'laughterAnnotation.csv')

            # Read the CSV file if it exists
            df_annotation = pd.read_csv(csv_file_path, encoding="ISO-8859-1")
            #Locate the rows where 'type' is true and select columns from index 1 to 3
            df_annotation = df_annotation.loc[(df_annotation['Type'] == 'Laughter') | (df_annotation['Type'] == 'PosedLaughter')].iloc[:, 1:4]

            # Concatenate it to the main annotations file+ as df
            df_annotations = pd.concat([df_annotations, df_annotation])
        else:
            print(f"Skipping {session_folder_path} as 'laughterAnnotation.csv' does not exist.")

        #Audio Processing.
        #Loop over each row in the df
        for row in np.array(df_annotation):
            start_time = row[1]
            end_time = row[2]
            #Load an audio file. Result is time series 'y' and sr
            y, sr = librosa.load(audio_file[0], offset=start_time, duration=(end_time-start_time), sr=None)

            if len(y) > num_samples:
              #Linear interpolation to downsample the audio
                downsampled_audio = [y[int(np.floor(i))] for i in np.linspace(0, len(y)-1, num_samples)]
            else:
              #pads the audio with zeros
                padded_zeros = [0 for i in range(0, num_samples-len(y))]
                downsampled_audio = list(y) + padded_zeros
            downsampled_audio = np.array(downsampled_audio)
            #Extract MFCC features and append them to ' input_data' list.
            MFCCs = librosa.feature.mfcc(y=downsampled_audio, sr=sr, n_mfcc=num_MFCC)
            input_data.append(MFCCs)

        os.chdir(data_folder)

Current Working Directory: /content/drive/MyDrive/data/Sessions/9
Files in Current Directory: ['S002-001_cam.wav', 'S002-001_synchPoints.csv', 'S002-001.eaf', 'laughterAnnotation.csv', 'S002-001.avi', 'session.xml', 'S002-001_mic.wav', 'S002-001_th.avi', 'S002-001_cam.wav_S002-001.mkv']
Current Working Directory: /content/drive/MyDrive/data/Sessions/11
Files in Current Directory: ['laughterAnnotation.csv', 'S002-003_1_cam.wav', 'S002-003_th_1.avi', 'S002-003_1.avi', 'S002-003_1_mic.wav', 'session.xml', 'S002-003_1.eaf', 'S002-003_synchPoints_1.csv', 'S002-003_1_mic.wav_S002-003_th_1.mkv']
Current Working Directory: /content/drive/MyDrive/data/Sessions/14
Files in Current Directory: ['S002-008.avi', 'S002-008_cam.wav', 'S002-008.eaf', 'S002-008_mic.wav', 'session.xml', 'laughterAnnotation.csv', 'S002-008_th.avi', 'S002-008_synchPoints.csv', 'S002-008_mic.wav_S002-008_th.mkv']
Current Working Directory: /content/drive/MyDrive/data/Sessions/20
Files in Current Directory: ['S003-001_mic.wa

Convert processed audio input into HDF5 file

In [ ]:
# Convert input_data to NumPy array
x_train = np.array(input_data)

#Original shape is (number_of_samples, number_of_MFCC_features, number_of_timesteps)
# LSTM expects timesteps x features (216 x 20 MFCC features)
x_train = np.swapaxes(x_train, 1, 2)

# Write the data into an HDF5 File
h5f = h5py.File('/content/drive/MyDrive/data/Models/Audio_Features_DL.h5', 'w')
h5f.create_dataset('dataset_features', data=x_train)


<HDF5 dataset "dataset_features": shape (98, 216, 20), type "<f8">

In [ ]:
h5f.close()

In [ ]:
print("Shape of x_train_audio:", x_train.shape)

Shape of x_train_audio: (98, 216, 20)


VIDEO FEATURE PROCESSING

In [ ]:
import numpy as np
import pandas as pd
import math
import random
import matplotlib.pyplot as plt
import os
import shutil

import keras
#from keras.utils import np_utils as np_utils_keras

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Dropout, Flatten
from keras.layers import Conv3D
from keras.layers import ConvLSTM2D
from keras.layers import BatchNormalization

from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.vgg16 import VGG16, preprocess_input

from keras.preprocessing import image

import cv2  # Computer Vision
from moviepy.editor import *
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import imageio
!pip install imageio-ffmpeg
import time

import h5py

In [ ]:
# Keep randomness the same
np.random.seed(0)

fps = 25 # original video recorded at 25fps
num_frames_to_use = 25  # Only use 25 frames from the laughter segment
img_size = 70  #Downsample the frames to 70x70 each, originally its (576, 720)
num_classes = 5

In [ ]:
input_data = []

In [ ]:
# Save home directory
project_home_dir = os.getcwd()

In [ ]:
# Empty Annotations File
df_annotations = pd.DataFrame(index=range(0,0), columns=['Type', 'Start Time (sec)', 'End Time (sec)'])

main_labelling_annotations = []

# Create a folder to store subclips
target_folder = os.path.join(data_folder, "Subclips")
os.makedirs(target_folder, exist_ok=True)

In [ ]:
# Create a folder to store subclips
target1_folder = os.path.join(data_folder, "SubclipswithAudio")
os.makedirs(target1_folder, exist_ok=True)

processing .mkv files and saving them in SubclipswithAudio for labelling purpose

In [ ]:
# for directory, subdirs, files in os.walk(os.path.join(data_folder, "Sessions")):
#     subdirs = [int(i) for i in subdirs]
#     subdirs.sort()
#     subdirs = [str(i) for i in subdirs]

#     for subdir in subdirs:
#         session_folder_path = os.path.join(data_folder, directory, subdir)
#         os.chdir(session_folder_path)

#         video_file = [f for f in os.listdir('.') if (os.path.isfile(f) and '.mkv' in f)]

#         if 'laughterAnnotation.csv' in os.listdir('.'):
#             csv_file_path = os.path.join(session_folder_path, 'laughterAnnotation.csv')

#             df_annotation = pd.read_csv(csv_file_path, encoding="ISO-8859-1")
#             df_annotation = df_annotation.loc[(df_annotation['Type'] == 'Laughter') | (df_annotation['Type'] == 'PosedLaughter')].iloc[:, 1:4]

#             # Concatenate it to the main annotations file
#             df_annotations = pd.concat([df_annotations, df_annotation])

#             # Portion to test
#             df_temp = pd.read_csv('laughterAnnotation.csv', encoding="ISO-8859-1")
#             for index, row in enumerate(np.array(df_temp)):
#                 type = row[1]
#                 if type == 'Laughter' or type == 'PosedLaughter':
#                     main_labelling_annotations.append(video_file[0].strip('.mkv') + '-l' + str(index).zfill(3))

#             # Go through the annotations for this audio file only
#             for index, row in enumerate(np.array(df_annotation)):
#                 start_time = row[1]
#                 end_time = row[2]
#                 # Read each video and edit it
#                 try:
#                     if start_time > end_time:
#                       start_time, end_time = end_time, start_time  # Swap start_time and end_time if needed
#                     editted_vid = VideoFileClip(video_file[0]).subclip(max(start_time, 0), end_time)
#                 except (OSError, ValueError) as ve:
#                     print(f"Error processing {video_file[0]}: {ve}")
#                     continue


#                 # Write it back to the target folder
#                 editted_name = f"session_{subdir}_clip{start_time}_{end_time}.mp4"
#                 target_path = os.path.join(target1_folder, editted_name)

#                 try:
#                   editted_vid.write_videofile(target_path, codec="libx264", audio_codec="aac")
#                   print(f"Video clip saved: {target_path}")
#                 except Exception as e:
#                   print(f"Error saving video clip: {e}")

#                 editted_vid.close()
#                 time.sleep(1)

#         else:
#             print(f"Skipping {session_folder_path} as 'laughterAnnotation.csv' does not exist.")

PROCESSING .avi file and saving subclips

In [ ]:
for directory, subdirs, files in os.walk(os.path.join(data_folder, "Sessions")):
    subdirs = [int(i) for i in subdirs]
    subdirs.sort()
    subdirs = [str(i) for i in subdirs]

    for subdir in subdirs:
        session_folder_path = os.path.join(data_folder, directory, subdir)
        os.chdir(session_folder_path)
        #Reads video file with .avi extension
        video_file = [f for f in os.listdir('.') if (os.path.isfile(f) and '.avi' in f)]

        if 'laughterAnnotation.csv' in os.listdir('.'):
            csv_file_path = os.path.join(session_folder_path, 'laughterAnnotation.csv')

            df_annotation = pd.read_csv(csv_file_path, encoding="ISO-8859-1")
            df_annotation = df_annotation.loc[(df_annotation['Type'] == 'Laughter') | (df_annotation['Type'] == 'PosedLaughter')].iloc[:, 1:4]

            # Concatenate it to the main annotations file
            df_annotations = pd.concat([df_annotations, df_annotation])

            # Portion to process video clips
            df_temp = pd.read_csv('laughterAnnotation.csv', encoding="ISO-8859-1")
            for index, row in enumerate(np.array(df_temp)):
                type = row[1]
                if type == 'Laughter' or type == 'PosedLaughter':
                  #it appends a string to the list main_labelling_annotations.
                  #This string includes the name of the video file and a zero-padded index
                    main_labelling_annotations.append(video_file[0].strip('.avi') + '-l' + str(index).zfill(3))

            # Go through the annotations for this audio file only
            for index, row in enumerate(np.array(df_annotation)):
                start_time = row[1]
                end_time = row[2]
                # Read each video and edit it
                try:
                    if start_time > end_time:
                      start_time, end_time = end_time, start_time
                    #It loads a video clip using the VideoFileClip function from the moviepy library
                    #subclipping it based on the start and end times.
                    editted_vid = VideoFileClip(video_file[0]).subclip(max(start_time, 0), end_time)
                except (OSError, ValueError) as ve:
                    print(f"Error processing {video_file[0]}: {ve}")
                    continue


                # Write it back to the target folder
                editted_name = f"session_{subdir}_clip{start_time}_{end_time}.mp4"
                target_path = os.path.join(target_folder, editted_name)

                try:
                  editted_vid.write_videofile(target_path, codec="libx264", audio_codec="aac")
                  print(f"Video clip saved: {target_path}")
                except Exception as e:
                  print(f"Error saving video clip: {e}")

                editted_vid.close()
                time.sleep(1)

                #It uses OpenCV to capture video frames, resize them, and store them in image_frames.
                vidcap = cv2.VideoCapture(target_path)
                num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
                image_frames = [cv2.resize(vidcap.read()[1], (img_size, img_size)) for i in range(0, int(num_frames))]


                # If the number of frames is bigger than the desired number of frames, downsample it
                if len(image_frames) > num_frames_to_use:
                    # Downsample the frames to 25 frames using linear interpolation
                    downsampled_frames = [image_frames[int(np.floor(i))] for i in np.linspace(0, len(image_frames)-1, num_frames_to_use)]

                # Else Just pad the end with zeros (black frames)
                else:
                    w, h, c = image_frames[0].shape  # width, height, channel
                    padded_frames = [np.zeros((w, h, c)) for i in range(0, num_frames_to_use-len(image_frames))]
                    downsampled_frames = image_frames + padded_frames

                downsampled_frames = np.array(downsampled_frames)
                input_data.append(downsampled_frames)
        else:
            print(f"Skipping {session_folder_path} as 'laughterAnnotation.csv' does not exist.")

Moviepy - Building video /content/drive/MyDrive/data/Subclips/session_9_clip0.576_7.542.mp4.
Moviepy - Writing video /content/drive/MyDrive/data/Subclips/session_9_clip0.576_7.542.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/data/Subclips/session_9_clip0.576_7.542.mp4
Video clip saved: /content/drive/MyDrive/data/Subclips/session_9_clip0.576_7.542.mp4
Moviepy - Building video /content/drive/MyDrive/data/Subclips/session_9_clip9.636_13.038.mp4.
Moviepy - Writing video /content/drive/MyDrive/data/Subclips/session_9_clip9.636_13.038.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/data/Subclips/session_9_clip9.636_13.038.mp4
Video clip saved: /content/drive/MyDrive/data/Subclips/session_9_clip9.636_13.038.mp4
Moviepy - Building video /content/drive/MyDrive/data/Subclips/session_9_clip17.226_31.812.mp4.
Moviepy - Writing video /content/drive/MyDrive/data/Subclips/session_9_clip17.226_31.812.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/data/Subclips/session_9_clip17.226_31.812.mp4
Video clip saved: /content/drive/MyDrive/data/Subclips/session_9_clip17.226_31.812.mp4
Moviepy - Building video /content/drive/MyDrive/data/Subclips/session_9_clip33.828_61.96.mp4.
Moviepy - Writing video /content/drive/MyDrive/data/Subclips/session_9_clip33.828_61.96.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/data/Subclips/session_9_clip33.828_61.96.mp4
Video clip saved: /content/drive/MyDrive/data/Subclips/session_9_clip33.828_61.96.mp4
Moviepy - Building video /content/drive/MyDrive/data/Subclips/session_9_clip63.75_64.92.mp4.
Moviepy - Writing video /content/drive/MyDrive/data/Subclips/session_9_clip63.75_64.92.mp4



...

t:   9%|▊         | 3/35 [00:00<00:03,  8.16it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file S013-004_th.avi, 746496 bytes wanted but 0 bytes read,at frame 204/205, at time 8.16/8.20 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  51%|█████▏    | 18/35 [00:00<00:00, 47.11it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file S013-004_th.avi, 746496 bytes wanted but 0 bytes read,at frame 209/205, at time 8.36/8.20 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.wa

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/data/Subclips/session_89_clip7.71_9.102.mp4
Video clip saved: /content/drive/MyDrive/data/Subclips/session_89_clip7.71_9.102.mp4
Error processing S013-004_th.avi: t_start (11.38) should be smaller than the clip's duration (8.20).
Moviepy - Building video /content/drive/MyDrive/data/Subclips/session_90_clip0.87_3.714.mp4.
Moviepy - Writing video /content/drive/MyDrive/data/Subclips/session_90_clip0.87_3.714.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/data/Subclips/session_90_clip0.87_3.714.mp4
Video clip saved: /content/drive/MyDrive/data/Subclips/session_90_clip0.87_3.714.mp4
Moviepy - Building video /content/drive/MyDrive/data/Subclips/session_91_clip0.336_2.43.mp4.
Moviepy - Writing video /content/drive/MyDrive/data/Subclips/session_91_clip0.336_2.43.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/data/Subclips/session_91_clip0.336_2.43.mp4
Video clip saved: /content/drive/MyDrive/data/Subclips/session_91_clip0.336_2.43.mp4
Moviepy - Building video /content/drive/MyDrive/data/Subclips/session_102_clip1.23_2.058.mp4.
Moviepy - Writing video /content/drive/MyDrive/data/Subclips/session_102_clip1.23_2.058.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/data/Subclips/session_102_clip1.23_2.058.mp4
Video clip saved: /content/drive/MyDrive/data/Subclips/session_102_clip1.23_2.058.mp4


In [ ]:
folder_path = '/content/drive/MyDrive/data/Subclips'
files_in_folder = os.listdir(folder_path)
num_files = len(files_in_folder)
print("Number of files in the folder:", num_files)


Number of files in the folder: 94


In [ ]:
print(len(input_data))

94


In [ ]:
# A single video/data sample is 4D: (frames, width, height, channels)  Our channel is 1 for grayscale
x_train = np.array(input_data)

# Scale and normalize the values, divide by 255
x_train = x_train/255


# Initialize success to the result of the first read
success, image = vidcap.read()
# image * 255 if the pixel values have been scaled
count = 0
#reads frames from the video, resizes, converts the frame to grayscale, reshapes it to 3D and saves it as a JPEG
while success:
	image = cv2.resize(image, (img_size, img_size))
	image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	image = image.reshape(img_size,img_size,1)   #no difference
	cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file
	success,image = vidcap.read() #read the next frame
	count += 1



# CROP the faces to improve the model
cropped_x_train = []

for video in x_train:
	frames = []
	#It creates a directory for each frame and saves the frame as a JPEG file.
	for ix, frame in enumerate(video):
		os.mkdir('./'+ str(ix))
		cv2.imwrite('./' + str(ix) + '/' + str(ix) +'.jpg', frame*255)
		#It uses the autocrop command-line tool to automatically crop faces from each saved image.
		cmd = 'yes | autocrop -i ' + './' + str(ix) + ' -o ./' + str(ix) + ' -w 70 -H 70'
		subprocess.call(cmd, shell=True)
		#The cropped frames are then read and appended to the frames list
		frames.append( cv2.imread('./' + str(ix) + '/' + str(ix) +'.jpg',1)  )
		shutil.rmtree('./' + str(ix))
	cropped_x_train.append( np.array(frames) )


cropped_x_train = np.array(cropped_x_train)

# Scale and normalize the values, divide by 255
cropped_x_train = cropped_x_train/255

In [ ]:
# Create and write to the first HDF5 file
h5f1 = h5py.File('/content/drive/MyDrive/data/Models/Video_Features_RGB_DL.h5', 'w')
h5f1.create_dataset('dataset_features', data=x_train)
h5f1.close()

# Create and write to the second HDF5 file
h5f2 = h5py.File('/content/drive/MyDrive/data/Models/Cropped_Video_Features_RGB_DL.h5', 'w')
h5f2.create_dataset('dataset_features', data=cropped_x_train)
h5f2.close()

# # Create and write to the third HDF5 file
# h5f3 = h5py.File('/content/drive/MyDrive/data/Models/Video_Features_Grayscale_DL.h5', 'w')
# h5f3.create_dataset('dataset_features', data=grayscale_x_train)
# h5f3.close()


In [ ]:
print("Shape of x_train_audio:", x_train.shape)
print("Shape of cropeped_x_train_audio:",cropped_x_train.shape)
#print("Shape of x_train_audio:",grayscale_x_train.shape)

Shape of x_train_audio: (94, 25, 70, 70, 3)
Shape of cropeped_x_train_audio: (94, 25, 70, 70, 3)


AUDIO VIDEO EMOTION

In [ ]:
import numpy as np
import h5py
import pandas as pd
import math
import random
import matplotlib.pyplot as plt
import os
import itertools

import keras


from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Dropout, Bidirectional, Input, concatenate, Flatten
from keras.layers import Permute, Reshape
from keras.layers import Conv3D
from keras.layers import ConvLSTM2D
from keras.layers import BatchNormalization

from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


In [ ]:
# Keep randomness the same
np.random.seed(0)


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
#pd.set_option('display.height', 1000)
pd.set_option('display.width', 1000)

In [ ]:
print("Files in the Directory:", os.listdir('/content/drive/MyDrive/data/'))

Files in the Directory: ['Sessions', 'Models', '.ipynb_checkpoints', 'output_files', 'SubclipswithAudio', 'Subclips', 'merged_labels.csv', 'audio_subclips']


In [ ]:
df_labels = pd.read_csv('/content/drive/MyDrive/data/merged_labels.csv',  encoding="ISO-8859-1")

In [ ]:
def labelling(x):
	if x == 'happy':
		return 0
	elif x == 'schadenfreude':
		return 1
	elif x == 'embarassment':
		return 2
	elif x == 'courtesy':
		return 3

In [ ]:
missing_values = df_labels['Merged Emotion'].isnull().sum()
print(missing_values)

0


In [ ]:
df_labels['Merged Emotion'] = df_labels['Merged Emotion'].apply(lambda x:labelling(x)  )
label_counts = df_labels['Merged Emotion'].value_counts()

# Print the label counts
print(label_counts)

0    42
3    33
1    17
2     6
Name: Merged Emotion, dtype: int64


In [ ]:
train_labels = np.array( df_labels['Merged Emotion'] )

In [ ]:
# Print a sample of the encoded labels
print(train_labels[:20])  # Print the first 10 labels as an example

[1 1 1 0 0 0 1 0 1 0 0 0 1 3 2 3 3 0 3 0]


In [ ]:
# Loading the File
h5f = h5py.File('/content/drive/MyDrive/data/Models/Audio_Features_DL.h5','r')
group_name = list(h5f.keys())[0]
x_train_audio = h5f[group_name][:]
h5f.close()

In [ ]:
len(df_labels['Merged Emotion'])

98

In [ ]:
print("Shape of x_train_audio:", x_train_audio.shape)
print("Length of train_labels:", len(train_labels))

Shape of x_train_audio: (98, 216, 20)
Length of train_labels: 98


In [ ]:
num_classes = len(np.unique(train_labels))
print(num_classes)

4


In [ ]:
####################################### Audio Model  ##################################################

# LSTM expects timesteps x features  (ie. 216 x 20 MFCC features)
_, timesteps, data_dim = x_train_audio.shape

x_train, x_cv, y_train_labels, y_cv_labels = train_test_split(x_train_audio, train_labels, test_size=0.20, random_state=42)

y_train = keras.utils.to_categorical(y_train_labels, len(np.unique(train_labels)))
y_cv = keras.utils.to_categorical(y_cv_labels, len(np.unique(train_labels)))

# define hyperparameters for the model
num_hidden_lstm = 128
batch_size = 128
epochs = 100
model_patience = 10

# LSTM expects 3D data (batch_size, timesteps, features)
model_audio = Sequential()
model_audio.add(BatchNormalization(input_shape=(timesteps, data_dim)))
model_audio.add(LSTM(num_hidden_lstm, return_sequences=True))
model_audio.add(BatchNormalization())
model_audio.add(LSTM(num_hidden_lstm, return_sequences=False))
model_audio.add(Dense(len(np.unique(train_labels)), activation='softmax'))

model_audio.summary()

model_audio.compile(
    loss=keras.losses.categorical_crossentropy,   # For emotional expression
    metrics=['accuracy'], 						# For emotional expression
    optimizer='adam'
)

checkpoint = ModelCheckpoint(filepath="audio_weights_emotion.hdf5", monitor='val_loss', verbose=1, save_best_only=True)

# Removed EarlyStopping callback from callbacks list
history = model_audio.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    shuffle=True,
    validation_data=(x_cv, y_cv),
    callbacks=[checkpoint]  # Removed EarlyStopping callback
)

model_audio.load_weights("audio_weights_emotion.hdf5")
model_audio.save('/content/drive/MyDrive/data/Models/audio_model_emotion.h5')

# Predict the scores
predicted_audio_labels_probs = model_audio.predict(x_cv)
predicted_audio_labels = np.argmax(predicted_audio_labels_probs, axis=1)

# Evaluation Metric (Emotion)
print('Test Accuracy: ', sum(predicted_audio_labels == y_cv_labels) / y_cv_labels.shape[0])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (Batch  (None, 216, 20)           80        
 Normalization)                                                  
                                                                 
 lstm (LSTM)                 (None, 216, 128)          76288     
                                                                 
 batch_normalization_1 (Bat  (None, 216, 128)          512       
 chNormalization)                                                
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 4)                 516       
                                                                 
Total params: 208980 (816.33 KB)
Trainable params: 20868

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - ETA: 0s - loss: 1.2251 - accuracy: 0.4615
Epoch 2: val_loss improved from 1.45898 to 1.42320, saving model to audio_weights_emotion.hdf5
1/1 [==============================] - 1s 1s/step - loss: 1.2251 - accuracy: 0.4615 - val_loss: 1.4232 - val_accuracy: 0.3000
Epoch 3/100
1/1 [==============================] - ETA: 0s - loss: 1.1293 - accuracy: 0.5769
Epoch 3: val_loss improved from 1.42320 to 1.40573, saving model to audio_weights_emotion.hdf5
1/1 [==============================] - 1s 1s/step - loss: 1.1293 - accuracy: 0.5769 - val_loss: 1.4057 - val_accuracy: 0.2500
Epoch 4/100
1/1 [==============================] - ETA: 0s - loss: 1.0575 - accuracy: 0.6026
Epoch 4: val_loss did not improve from 1.40573
1/1 [==============================] - 1s 1s/step - loss: 1.0575 - accuracy: 0.6026 - val_loss: 1.4061 - val_accuracy: 0.1000
Epoch 5/100
1/1 [==============================] - ETA: 0s - loss: 0.9998 - accuracy: 0.6154
Epoch 5: val_loss did not

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Calculate accuracy, precision, and recall
accuracy = accuracy_score(y_cv_labels, predicted_audio_labels)
precision = precision_score(y_cv_labels, predicted_audio_labels, average='weighted')
recall = recall_score(y_cv_labels, predicted_audio_labels, average='weighted')
f1 = f1_score(y_cv_labels, predicted_audio_labels, average='weighted')
final_training_accuracy = history.history['accuracy'][-1]

print('Overall Training Accuracy:', final_training_accuracy)
print('Test Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.




Overall Training Accuracy: 0.7179487347602844
Test Accuracy: 0.3
Precision: 0.28823529411764703
Recall: 0.3
F1 Score: 0.2104347826086957


In [ ]:
import plotly.graph_objects as go

# Plot the training and validation accuracy and loss using Plotly
fig = go.Figure()

# Add training accuracy trace
fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=history.history['accuracy'], mode='lines', name='Training Accuracy'))

# Add validation accuracy trace
fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=history.history['val_accuracy'], mode='lines', name='Validation Accuracy'))

# Add training loss trace
fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=history.history['loss'], mode='lines', name='Training Loss'))

# Add validation loss trace
fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=history.history['val_loss'], mode='lines', name='Validation Loss'))

# Update layout
fig.update_layout(
    title='Model Accuracy and Loss',
    xaxis_title='Epoch',
    yaxis_title='Value',
    legend=dict(
        orientation='h',
        yanchor='top',
        y=1.1,
        xanchor='right',
        x=1
    )
)

# Show the plot
fig.show()

In [ ]:
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix

# Assuming you have imported your true labels and predicted labels as y_cv_labels and predicted_audio_labels respectively

# Compute confusion matrix
cm =confusion_matrix(y_cv_labels,predicted_audio_labels).T

# Plot confusion matrix using Plotly
fig = ff.create_annotated_heatmap(z=cm,
                                  x=['Courtesy','Embarassment','Happy','Schadenfreude'] ,
                                  y=['Schadenfreude', 'Happy','Embarassment', 'Courtesy'] ,
                                  colorscale='Blues')
# Update the layout to reduce the size
fig.update_layout(
    title='Confusion Matrix',
    xaxis=dict(title='Predicted label'),
    yaxis=dict(title='True label'),
    width=600,
    height=500,  # Adjust the height as needed
)
fig.show()


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
# Get the classification report
class_report = classification_report(y_cv_labels, predicted_audio_labels)

# Print the classification report
print("\nClassification Report:")
print(class_report)


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.





Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.12      0.20         8
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         1
           3       0.29      0.83      0.43         6

    accuracy                           0.30        20
   macro avg       0.20      0.24      0.16        20
weighted avg       0.29      0.30      0.21        20



In [ ]:
import librosa
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model_audio = load_model("/content/drive/MyDrive/data/Models/audio_model_emotion.h5")

# Load and preprocess the audio file
audio_file_path = '/content/drive/MyDrive/data/audio_subclips/session_102/subclip_0.wav'  # Path to your audio file
y, sr = librosa.load(audio_file_path, sr=None)  # Load audio without resampling
MFCCs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=num_MFCC)

# Ensure the number of timesteps matches the expected shape (216)
if MFCCs.shape[1] < 216:
    # If fewer timesteps, pad the MFCCs with zeros to match the expected shape
    MFCCs = np.pad(MFCCs, ((0, 0), (0, 216 - MFCCs.shape[1])), mode='constant')
elif MFCCs.shape[1] > 216:
    # If more timesteps, truncate the MFCCs to match the expected shape
    MFCCs = MFCCs[:, :216]

# Reshape the MFCCs to match the model's input shape
MFCCs = np.expand_dims(MFCCs, axis=0)  # Add batch dimension
MFCCs = np.swapaxes(MFCCs, 1, 2)  # Swap axes to get timesteps x features format

# Predict the emotion label
predicted_probabilities = model_audio.predict(MFCCs)
predicted_label_index = np.argmax(predicted_probabilities)
emotion_labels = ['happy', 'schadenfreude', 'embarrassment', 'courtesy']
predicted_emotion = emotion_labels[predicted_label_index]
print("Predicted Emotion:", predicted_emotion)


1/1 [==============================] - 1s 1s/step
Predicted Emotion: happy


In [ ]:
####################################### Video Model  ##################################################

# # h5f = h5py.File('Video_Features_RGB_DL.h5','r')
# h5f = h5py.File('Cropped_Video_Features_RGB_DL.h5','r')
# # h5f = h5py.File('Video_Features_Grayscale_DL.h5','r')
# group_name = list(h5f.keys())[0]
# x_train_video = h5f[group_name][:]
# h5f.close()

h5f = h5py.File('/content/drive/MyDrive/data/Models/Video_Features_Grayscale_DL.h5','r')

# Print the keys to see what's in the file
print("Keys in the HDF5 file:", list(h5f.keys()))

# Check if there are any keys before trying to access the first one
if len(h5f.keys()) > 0:
    group_name = list(h5f.keys())[0]
    x_train_video = h5f[group_name][:]
    h5f.close()
else:
    print("No keys found in the HDF5 file.")

# x_train_video.shape: (614, 25, 70, 70, 1)
frames = x_train_video.shape[1]
width = x_train_video.shape[2]
height = x_train_video.shape[3]
channels = x_train_video.shape[4]  # Grayscale (1) or RGB (3)

num_hidden_lstm = 128
num_hidden_units = 256
batch_size = 128
epochs = 50
model_patience = 10


model_vgg16 = VGG16(weights='imagenet', include_top=False)  # Feature extracted is (2, 2, 512) for each input (video frame)

x_train_video_features = []

# Loop through the video training data to extract features for each frame
for video in x_train_video:
	# frame is (70,70,1) or (70,70,3) depending on features
	# model expects (1,70,70,3)  -  The RGB version
	video_features = [model_vgg16.predict(np.expand_dims(frame, axis=0)).reshape(2048,-1) for frame in video]
	x_train_video_features.append(video_features)


x_train_video_features = np.array(x_train_video_features)  # (num_laughters, num_frames (25), features extracted (2048), 1)
x_train_video_features = x_train_video_features.reshape(len(x_train_video_features), 25, 2048)  #2048 features for each of the 25 frames


train_labels = np.array( df_labels['Merged Emotion'] )

x_train, x_cv, y_train_labels, y_cv_labels = train_test_split(x_train_video_features, train_labels, test_size=0.20, random_state=42)

y_train = keras.utils.to_categorical( y_train_labels, len(np.unique(train_labels)))
y_cv = keras.utils.to_categorical( y_cv_labels, len(np.unique(train_labels)))


# LSTM expects timesteps x features  (ie. 25 x 2048 Frame features)
_, timesteps, data_dim = x_train_video_features.shape

model_video = Sequential()
# model_video.add(Flatten(input_shape=(25, 2048)))
# model_video.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_video.add(LSTM(num_hidden_lstm, input_shape=(timesteps,data_dim), return_sequences=True))  # returns a sequence of vectors of dimension 128
model_video.add(BatchNormalization())
model_video.add(LSTM(num_hidden_lstm, return_sequences=False))  # return a single vector of dimension 128
model_video.add(Dense(len(np.unique(train_labels)), activation='softmax'))

model_video.summary()

model_video.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='Adam',
              metrics=['accuracy'])

checkpoint = ModelCheckpoint(filepath="video_weights_noncropped_emotion.hdf5", monitor='val_loss', verbose=1, save_best_only=True)
checkpoint = ModelCheckpoint(filepath="video_weights_cropped_emotion.hdf5", monitor='val_acc', verbose=1, save_best_only=True)  # Val acc for cropped version

model_video.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_cv, y_cv),
            shuffle=True,
            callbacks = [checkpoint, EarlyStopping(monitor='val_loss', patience= model_patience)])  # 10 epochs patience

model_video.load_weights("video_weights_noncropped_emotion.hdf5")
model_video.load_weights("video_weights_cropped_emotion.hdf5")


# Predict the scores
predicted_video_labels = np.argmax(model_video.predict(x_cv), axis=1)
predicted_video_labels_probs = model_video.predict(x_cv)

# Evaluation Metric (Emotion)
print ('Test Accuracy: ', sum(predicted_video_labels == y_cv_labels)/y_cv_labels.shape[0])
confusion_matrix(y_cv_labels,predicted_video_labels).T
print (metrics.classification_report(y_cv_labels, predicted_video_labels))


Keys in the HDF5 file: []
No keys found in the HDF5 file.


NameError: name 'x_train_video' is not defined

In [ ]:
####################################### Ensemble Model  ##################################################

# Average them out equally and try unequal weighting too
ensemble_predictions_probs = (predicted_video_labels_probs+predicted_audio_labels_probs)/2  # Weighted equally
ensemble_predictions_labels = np.argmax(ensemble_predictions_probs, axis=1)


# Evaluation Metric (Emotion)
print ('Test Accuracy: ', sum(ensemble_predictions_labels == y_cv_labels)/y_cv_labels.shape[0])
confusion_matrix(y_cv_labels,ensemble_predictions_labels).T
print (metrics.classification_report(y_cv_labels, ensemble_predictions_labels))



# Unequally weighted ensemble
ensemble_weights = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

best_ensemble_accuracy = 0.0
best_ensemble_audio_weight = 0.0
best_ensemble_video_weight = 0.0

for i in zip(ensemble_weights, ensemble_weights[::-1]):
	audio_weight, video_weight = i  # i is a tuple (sum of the weights should be 1)
	prediction_probs = (audio_weight * predicted_audio_labels_probs) + (video_weight * predicted_video_labels_probs)
	prediction_probs_labels = np.argmax(prediction_probs, axis=1)
	ensemble_accuracy = sum(prediction_probs_labels == y_cv_labels)/y_cv_labels.shape[0]
	if ensemble_accuracy > best_ensemble_accuracy:
		best_ensemble_audio_weight = audio_weight
		best_ensemble_video_weight = video_weight
		best_ensemble_accuracy = ensemble_accuracy


ensemble_predictions_probs = (best_ensemble_audio_weight * predicted_audio_labels_probs) + (best_ensemble_video_weight * predicted_video_labels_probs)  # Weighted Unequally
ensemble_predictions_labels = np.argmax(ensemble_predictions_probs, axis=1)

# Evaluation Metric (Emotion)
print ('Test Accuracy: ', sum(ensemble_predictions_labels == y_cv_labels)/y_cv_labels.shape[0])
confusion_matrix(y_cv_labels,ensemble_predictions_labels).T
print (metrics.classification_report(y_cv_labels, ensemble_predictions_labels))


In [ ]:
import numpy as np
import pandas as pd
import h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC  # You can replace this with your preferred model

# Load combined audio and video features
audio_features_file = '/content/drive/MyDrive/data/Models/Audio_Features_DL.h5'
video_features_file = '/content/drive/MyDrive/data/Models/Video_Features_RGB_DL.h5'

with h5py.File(audio_features_file, 'r') as audio_file:
    audio_features = audio_file['dataset_features'][:]

with h5py.File(video_features_file, 'r') as video_file:
    video_features = video_file['dataset_features'][:]

# Assuming you have labels available in train_labels (e.g., Merged Arousal, Merged Dominance, Merged Emotion)
# Replace this with the actual labels you want to predict
train_labels_arousal = np.array(df_labels['Merged Arousal'])
train_labels_dominance = np.array(df_labels['Merged Dominance'])
train_labels_emotion = np.array(df_labels['Merged Emotion'])

# Combine audio and video features
combined_features = np.concatenate((audio_features, video_features), axis=-1)

# Split the data for training/testing
X_train, X_test, y_arousal_train, y_arousal_test, y_dominance_train, y_dominance_test, y_emotion_train, y_emotion_test = train_test_split(
    combined_features, train_labels_arousal, train_labels_dominance, train_labels_emotion, test_size=0.2, random_state=42
)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train or load your model (using SVM as an example)
arousal_model = SVC()
dominance_model = SVC()
emotion_model = SVC()

arousal_model.fit(X_train, y_arousal_train)
dominance_model.fit(X_train, y_dominance_train)
emotion_model.fit(X_train, y_emotion_train)

# Make predictions
y_arousal_pred = arousal_model.predict(X_test)
y_dominance_pred = dominance_model.predict(X_test)
y_emotion_pred = emotion_model.predict(X_test)

predictions_df = pd.DataFrame({
    'Arousal_Predicted': y_arousal_pred,
    'Dominance_Predicted': y_dominance_pred,
    'Emotion_Predicted': y_emotion_pred
})

# Save the predictions to a CSV file
predictions_df.to_csv('/content/drive/MyDrive/data/predictions.csv', index=False)